# Text classification model

In this notebook, we will load and use a text classification language model that can assign a label to a given text. Learn more about the Text Classification task <a href="https://huggingface.co/tasks/text-classification" target="_blank" rel="noopener">here</a>. Please also refer to the Transformer Extension <a href="https://github.com/exasol/transformers-extension/blob/main/doc/user_guide/user_guide.md" target="_blank" rel="noopener">User Guide</a> to find more information about the UDFs used in this notebook.

We will be running SQL queries using <a href="https://jupysql.ploomber.io/en/latest/quick-start.html" target="_blank" rel="noopener"> JupySQL</a> SQL Magic.

## Prerequisites

Prior to using this notebook the following steps need to be completed:
1. [Configure the sandbox](../sandbox_config.ipynb).
2. [Initialize the Transformer Extension](te_init.ipynb).

## Set up

### Access configuration

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

Let's bring up JupySQL and connect to the database via SQLAlchemy. 
Please refer to the documentation of [sqlalchemy-exasol](https://github.com/exasol/sqlalchemy-exasol) for details on how to connect to the database using the Exasol SQLAlchemy driver.

In [ ]:
%run ../utils/jupysql_init.ipynb

## Get language model

To demonstrate the text classification task we will use the [Ekman emotions classifier](https://huggingface.co/arpanghoshal/EkmanClassifier) model.

We need to load the model from the Huggingface hub into the BucketFS. This could potentially be a long process. Unfortunately, we cannot tell exactly when it has finished. The notebook's hourglass may not be a reliable indicator. BucketFS will still be doing some work when the call issued by the notebook returns. Please wait for a few moments after that, before querying the model.

In [ ]:
# This is the name of the model at the Huggingface Hub
MODEL_NAME = 'arpanghoshal/EkmanClassifier'

In [ ]:
%run utils/model_retrieval.ipynb
load_huggingface_model(MODEL_NAME, sb_config, method='udf')

## Use language model

Let's try to classify a single phrase which definitely bears emotions but is also somewhat ambiguous - "Oh my God!"

In [ ]:
%%sql
WITH MODEL_OUTPUT AS
(
    SELECT TE_SEQUENCE_CLASSIFICATION_SINGLE_TEXT_UDF(
        NULL,
        '{{sb_config.TE_BFS_CONN}}',
        '{{sb_config.TE_TOKEN_CONN}}',
        '{{sb_config.TE_MODELS_BFS_DIR}}',
        '{{MODEL_NAME}}',
        'Oh my God!'
    )
)
SELECT label, score, error_message FROM MODEL_OUTPUT  ORDER BY SCORE DESC

Now we are going to add some context to our exclamation and use another UDF that takes a pair of sentences. Let's see how it will change the model output.

In [ ]:
%%sql
WITH MODEL_OUTPUT AS
(
    SELECT TE_SEQUENCE_CLASSIFICATION_TEXT_PAIR_UDF(
        NULL,
        '{{sb_config.TE_BFS_CONN}}',
        '{{sb_config.TE_TOKEN_CONN}}',
        '{{sb_config.TE_MODELS_BFS_DIR}}',
        '{{MODEL_NAME}}',
        'Oh my God!',
        'I lost my purse.'
    )
)
SELECT label, score, error_message FROM MODEL_OUTPUT ORDER BY SCORE DESC

The code above shows how the model works on a toy example. However, the main purpose of having a model deployed in the database is to get a quick response for a batch input. The performance gain comes from two factors - localization and parallelization. The first means that the input data never crosses the machine boundaries. The second means that multiple instances of the model are processing the data on all available nodes in parallel.

Another advantage of making predictions within the database is enhanced data security. The task of safeguarding privacy can be simplified given the fact that the source data never leaves the database machine.

In a more practical application, the text to be classified would be stored in a column of a database table. For example, if we wanted to get a label with the highest score for each row of the input table `MY_TEXT_TABLE`, where the text in question is in the column `MY_TEXT_COLUMN`, the SQL would look similar to this:
```
SELECT TE_SEQUENCE_CLASSIFICATION_SINGLE_TEXT_UDF(..., MY_TEXT_COLUMN) FROM MY_TEXT_TABLE;
```
Please note, that the response time observed on the provided example with a single input will not be scaled up linearly in case of multiple inputs. Much of the latency falls on loading the model into the CPU memory from BucketFS. This needs to be done only once regardless of the number of inputs.